In [1]:
import requests

In [2]:
url='https://www.google.es'

In [3]:
r1=requests.get(url)
r1.status_code

200

In [4]:
r2=requests.get(url+'/forbidden')
r2.status_code

404

In [5]:
r3=requests.get('https://www.google.com')
r3.history

[]

In [6]:
r3.history[0]

IndexError: list index out of range

In [ ]:
# Asyncio

import asyncio

In [ ]:
url='https://s3-eu-west-1.amazonaws.com/'
requests.get(url)

In [ ]:
urls = [
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/breakfast.jpg',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html'
]

In [ ]:
async def main():
    loop=asyncio.get_event_loop()
    
    futures=[loop.run_in_executor(None, requests.get, url) for url in urls]
    
    for res in await asyncio.gather(*futures):
        print (res.status_code)
        
loop=asyncio.get_event_loop()
loop.run_until_complete(main())

In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [ ]:
def get_page(url):
    driver=webdriver.Firefox()
    driver.get(url)
    html=driver.page_source
    
    try:
        soup=bs(html, 'html.parser')
        
    except:
        print ('Error en la sopa..')
        
    driver.quit()
    return soup

In [ ]:
soup=get_page('https://www.ironhack.com/es')
#soup

In [ ]:
# con clase

class PageGetter():
    driver=False
    def __init__(self, browser='firefox'):
        if browser=='firefox':
            self.driver=webdriver.Firefox()
        elif browser=='chrome':
            self.driver=webdriver.Chrome()
        else:
            raise Exception ('Browser Not Valid.')
            
            
    def get_page(self, url):
        if not self.driver:
            raise Exception ('Select a browser...')

        self.driver.get(url)
        html=self.driver.page_source
        soup=bs(html, 'html.parser')
        return soup
    
    def close(self):
        self.driver.quit()

In [ ]:
pg=PageGetter('firefox')
soup=pg.get_page('https://www.ironhack.com/es')
pg.close()
#soup

In [ ]:
soup.select('article')

In [ ]:
from pyvirtualdisplay import Display
import time

In [ ]:
display=Display(visible=0)
display.start()

In [ ]:
url='https://threatbutt.com/map/'

In [ ]:
driver=webdriver.Firefox()

while 1:
    driver.get(url)
    time.sleep(3)
    count=0
    while count<300:
        elemento=driver.find_element_by_id('attackdiv')
        t=elemento.text.split('\n')[-1]
        
        predator=(t.split('('))[0]
        ip_pred=(t.split('('))[1].split(')')[0]
        
        prey=(t.split('('))[1].split(')')[1][-4:]
        ip_prey=(t.split('('))[2].split(')')[0]
        
        print ({'timestamp':time.time(), 'predator':predator, 'ip_pred':ip_pred,
                'prey':prey, 'ip_prey':ip_prey})
        count+=1

In [ ]:
import pymongo
import re

In [ ]:
client=pymongo.MongoClient()
db=client.geo_attack
geo=list(db.geo2.find())

list_ip=[e['ip_pred'] for e in geo]

In [ ]:
for i in range(0, len(list_ip), 50):
    
    try:
        res=requests.get('https://tools.keycdn.com/geo?host={}'.format(list_ip[i])).content
        soup=bs(res, features='lxml')
        elem=soup.find_all('dd', {'class':'col-8 text-monospace'})
        
        for f in elem:
            F=str(f)
            
            if 'lat' in F:
                lat=re.findall('-?\d+.\d+', F.split('/')[0])[0]
                lng=re.findall('-?\d+.\d+', F.split('/')[1])[0]
                print ()
                print ('IP: {}'.format(list_ip[i]))
                print ({'latitude':lat, 'longitude':lng})
    except:
        continue

In [ ]:
# RSS


import requests as req
import xmltodict
import re
import pymongo 
import time



    
    
client=pymongo.MongoClient()
db=client.earthquake





def getxml():
    url = 'http://www.ssn.unam.mx/rss/ultimos-sismos.xml'
    data=req.get(url).content
    return xmltodict.parse(data)




    
def earth():

    d=getxml()

    for i in range(len(d['rss']['channel']['item'])):

        data_get=d['rss']['channel']['item'][i]['description'].split('>')[1:4]

        fecha=re.findall('\d+-\d+-\d+', data_get[0])[0]
        hora=re.findall('\d+:\d+:\d+', data_get[0])[0]
        lat=re.findall('-?\d+.\d+', data_get[1])[0]
        lon=re.findall('-?\d+.\d+', data_get[1])[1]
        prof=re.findall('\d+.\d+ \w+', data_get[2])[0]

        inten=float(d['rss']['channel']['item'][i]['title'].split(',')[0])

        data={'Fecha':fecha, 'Hora':hora,
              'Latitud':float(lat), 'Longitud':float(lon),
              'Profundidad':prof, 'Intensidad':inten}
        print (data)
        db.ssnUnam.update(data, data, upsert=True)





while 1:
    earth()
    print ('paso1')
    print ()
    time.sleep(10)
    print ('paso2')
    print ()
